## Data processing for dashboard

## Import libraries

In [70]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from dateutil.relativedelta import relativedelta
from datetime import datetime

#import matplotlib.pyplot as plt
#import seaborn as sns

#from sklearn.preprocessing import MinMaxScaler
#import geopandas as gpd

## Read in and process dependent variable - turnover

Read turnover data from csv into a dataframe.

In [71]:
# annual and monthly data

annual_url = '../01_data/processed_annual_turnover.csv'
monthly_url = '../01_data/processed_monthly_turnover.csv'

annual_df = pd.read_csv(annual_url, parse_dates=['month_year'])
annual_df = annual_df.drop(['n'],axis=1)
#annual_df.info()

monthly_df = pd.read_csv(monthly_url, parse_dates=['month_year'])
monthly_df = monthly_df.drop(['n'],axis=1)
monthly_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154588 entries, 0 to 154587
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   org_code     154588 non-null  object        
 1   region_name  154588 non-null  object        
 2   org_type     154588 non-null  object        
 3   staff_group  154588 non-null  object        
 4   month_year   154588 non-null  datetime64[ns]
 5   join_HC      115789 non-null  float64       
 6   join_FTE     115789 non-null  float64       
 7   leave_HC     115919 non-null  float64       
 8   leave_FTE    115919 non-null  float64       
 9   denom_HC     154258 non-null  float64       
 10  denom_FTE    154258 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 13.0+ MB


Function to clean and process DV dataframes

In [72]:
def clean_dv(df):
    # drop org_type because it has historic variation which is creating duplicates when mapping later
    df.drop('org_type', axis=1, inplace=True) 
    # add leaver and joiner rates column
    df['leaver_rate'] = df['leave_FTE']/df['denom_FTE']
    df['joiner_rate'] = df['join_FTE']/df['denom_FTE']

    # replace inf values with nan (can happen with rate calcs)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    return df

In [73]:
annual_df1 = clean_dv(annual_df)
monthly_df1 = clean_dv(monthly_df)

In [74]:
annual_df1.head()

,org_code,region_name,staff_group,month_year,join_HC,join_FTE,leave_HC,leave_FTE,denom_HC,denom_FTE,leaver_rate,joiner_rate
0,R0A,North West,All staff groups,2017-10-01,NaN,NaN,NaN,NaN,19130.0,17095.95832,NaN,NaN
1,R0A,North West,All staff groups,2017-11-01,NaN,NaN,NaN,NaN,19211.0,17171.25859,NaN,NaN
2,R0A,North West,All staff groups,2017-12-01,NaN,NaN,NaN,NaN,19214.0,17170.86679,NaN,NaN
3,R0A,North West,All staff groups,2018-01-01,NaN,NaN,NaN,NaN,19295.0,17240.69400,NaN,NaN
4,R0A,North West,All staff groups,2018-02-01,NaN,NaN,NaN,NaN,19326.0,17271.63829,NaN,NaN


The data show the full time equivalent (FTE) and headcount (HC) number of leavers by organisation and staff group for the previous 12-month period from the date.

In [75]:
annual_df1['staff_group'].unique()

array(['All staff groups', 'Ambulance staff', 'Central functions',
       'HCHS doctors (exc. junior Drs)', 'Hotel, property & estates',
       'Managers', 'Midwives', 'Nurses & health visitors',
       'Other staff or those with unknown classification',
       'Scientific, therapeutic & technical staff', 'Senior managers',
       'Support to ST&T staff', 'Support to doctors, nurses & midwives',
       'Support to ambulance staff'], dtype=object)

## Load independent variable 1 - local unemployment

Load data about local unemployment.

In [76]:
url_r1 = '../01_data/ONS_localunemployment_monthly.csv'
df_r1 = pd.read_csv(url_r1, parse_dates=['Date'])

df_r1.drop(['Region','thousands'],axis=1,inplace=True)
df_r1.rename(columns={'%':'local_unemployment','Date':'month_year',
                      'NHSE region name':'region_name'},inplace=True)
df_r1 = df_r1.sort_values('month_year')
df_r1.tail()

,month_year,region_name,local_unemployment
3369,2023-06-01,North East,0.052
3370,2023-06-01,North West,0.053
3371,2023-06-01,South East,0.039
3372,2023-06-01,South West,0.036
3374,2023-06-01,North East and Yorkshire,0.029


## Load IV 2 - sickness absence

Load data about sickness absence to use as second regressor

In [77]:
url_r2 = '../01_data/sickness_absence.csv'
df_r2 = pd.read_csv(url_r2, parse_dates=['Date'])
trust_types_todrop = ['Clinical Commissioning Group','Integrated Care Board']
df_r2 = df_r2[~df_r2['Cluster group'].isin(trust_types_todrop)]
df_r2 = df_r2.drop(['Benchmark group','Org name',
                    'NHSE region code','Cluster group'],axis=1)
df_r2.rename(columns={'Org code':'org_code','Date':'month_year',
                      'NHSE region name':'region_name','Staff group':'staff_group',
                      'FTE days lost':'fte_days_lost','FTE days available':'fte_days_available'},inplace=True)
merge_cols = ['month_year', 'org_code','region_name','staff_group']
df_r2['sickness_absence'] = df_r2['fte_days_lost']/df_r2['fte_days_available']
df_r2 = df_r2.reset_index(drop=True)
df_r2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248498 entries, 0 to 248497
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   month_year          248498 non-null  datetime64[ns]
 1   org_code            248498 non-null  object        
 2   region_name         248498 non-null  object        
 3   staff_group         248498 non-null  object        
 4   fte_days_lost       248498 non-null  float64       
 5   fte_days_available  248498 non-null  float64       
 6   sickness_absence    248244 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 13.3+ MB


Create 12-month rolling sickness absence column for use with annual turnover data

In [78]:
df_r2['month_year'] = pd.to_datetime(df_r2['month_year'])

# Sort the DataFrame by organisation, staff_group, and month
df_r2.sort_values(by=['org_code', 'staff_group', 'month_year'], inplace=True)

# Calculate the rolling sums for days lost and days available
df_r2['rolling_days_lost'] = df_r2.groupby(['org_code', 
                        'staff_group'])['fte_days_lost'].rolling(window=12, min_periods=1).sum().reset_index(level=[0, 1], drop=True)

df_r2['rolling_days_available'] = df_r2.groupby(['org_code', 
                        'staff_group'])['fte_days_available'].rolling(window=12, min_periods=1).sum().reset_index(level=[0, 1], drop=True)

# Calculate the rolling sickness absence rate
df_r2['annual_sickness_absence'] = df_r2['rolling_days_lost'] / df_r2['rolling_days_available']

# drop fte_days_lost fte_days_available, rolling_days_available and rolling_days_lost columns
df_r2.drop(columns=['fte_days_lost', 'fte_days_available', 'rolling_days_available', 'rolling_days_lost'], inplace=True)

df_r2.head()

,month_year,org_code,region_name,staff_group,sickness_absence,annual_sickness_absence
1853,2018-03-01,0AR,Special Health Authorities and other statutory...,All staff groups,0.030707,0.030707
6340,2018-04-01,0AR,Special Health Authorities and other statutory...,All staff groups,0.038024,0.034318
12436,2018-05-01,0AR,Special Health Authorities and other statutory...,All staff groups,0.039730,0.036162
15963,2018-06-01,0AR,Special Health Authorities and other statutory...,All staff groups,0.033094,0.035407
20980,2018-07-01,0AR,Special Health Authorities and other statutory...,All staff groups,0.031692,0.034648


## Load IV 3 - reasons for sickness absence

Add data about reasons for sickness absence

In [79]:
url_r3 = '../01_data/sickness_absence_reason_pivot.csv'
df_r3 = pd.read_csv(url_r3, parse_dates=['Date'])
#df_r3 = df_r3.drop(['FTE days lost'],axis=1)
df_r3.rename(columns={'Date':'month_year','Staff group':'staff_group'},inplace=True)
#df_r2 = df_r2.reset_index(drop=True)

# drop least frequent reasons for absence
df_r3 = df_r3.drop(['substance_abus','asthma',
                    'dental','blood_disorder','endocrine',
                    'eye','skin_disorders','nervous_system'],axis=1)

# Replace NaN values with 0 
df_r3 = df_r3.fillna(0)
df_r3.info()
# national level data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413 entries, 0 to 1412
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   month_year         1413 non-null   datetime64[ns]
 1   staff_group        1413 non-null   object        
 2   anxiety_stress     1413 non-null   float64       
 3   back_problems      1413 non-null   float64       
 4   gastro             1413 non-null   float64       
 5   gynaecological     1413 non-null   float64       
 6   headache_mig       1413 non-null   float64       
 7   infectious_dis     1413 non-null   float64       
 8   other              1413 non-null   float64       
 9   other_msk          1413 non-null   float64       
 10  pregnancy_related  1413 non-null   float64       
 11  respiratory        1413 non-null   float64       
 12  unknown            1413 non-null   float64       
dtypes: datetime64[ns](1), float64(11), object(1)
memory usage: 143.

In [93]:
df_r3.tail()

,month_year,staff_group,anxiety_stress,back_problems,gastro,gynaecological,headache_mig,infectious_dis,other,other_msk,pregnancy_related,respiratory,unknown
1408,2022-06-01,"Scientific, therapeutic & technical staff",0.299827,0.039535,0.102062,0.042492,0.034684,0.112587,0.073708,0.086431,0.038768,0.093175,0.032900
1409,2022-06-01,Senior managers,0.381579,0.032083,0.050199,0.025234,0.015957,0.079350,0.138571,0.033597,0.023071,0.094360,0.088860
1410,2022-06-01,Support to ST&T staff,0.312630,0.057006,0.103472,0.033756,0.035771,0.083623,0.075481,0.108540,0.022243,0.081857,0.031112
1411,2022-06-01,Support to ambulance staff,0.319816,0.080901,0.098365,0.027831,0.031204,0.104323,0.056427,0.094098,0.020951,0.077216,0.042403
1412,2022-06-01,"Support to doctors, nurses & midwives",0.311573,0.065024,0.097089,0.042193,0.027871,0.070502,0.076314,0.125198,0.029533,0.071209,0.030315


## Load IV 4 and 5 - staff vacancies

In [80]:
url_sg_ref = '../01_data/ref_sg_vacancy.csv'
df_sg_ref = pd.read_csv(url_sg_ref)
df_sg_ref.head()

,staff_group,vacancy_sg,all
0,Ambulance staff,Allied Health Professionals,All staff groups
1,Central functions,Administrative and Clerical,All staff groups
2,HCHS doctors (exc. junior Drs),Medical and Dental,All staff groups
3,"Hotel, property & estates",Estates and Ancillary,All staff groups
4,Managers,Administrative and Clerical,All staff groups


In [81]:
url_r4 = '../01_data/vacancy_ESR.csv'
df_r4 = pd.read_csv(url_r4,parse_dates=['month_year'],dayfirst=True)

df_r4 = df_r4.drop(['Published month','Published quarter','England'],axis=1)
df_r4.rename(columns={'NWD Staff Group':'vacancy_sg','NHS England region':'region_name',
                        'Vacancy Wte':'vacancy_FTE'},inplace=True)

df_r4 = df_r4.fillna(0)

# Remove code in brackets
df_r4['region_name'] = df_r4['region_name'].str[:-6].str.rstrip()

# Add staff groupings to match other datasets
df_r4 = pd.merge(df_r4, df_sg_ref, on='vacancy_sg',how='left')

df_r4 = df_r4.drop(['all'],axis=1)

df_r4.info()

# regional level

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8387 entries, 0 to 8386
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   month_year   8387 non-null   datetime64[ns]
 1   vacancy_sg   8387 non-null   object        
 2   region_name  8387 non-null   object        
 3   vacancy_FTE  8387 non-null   float64       
 4   staff_group  7948 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 327.7+ KB


In [82]:
url_r5 = '../01_data/vacancy_TRAC.csv'
df_r5 = pd.read_csv(url_r5,parse_dates=['month_year'],dayfirst=True)

df_r5 = df_r5.drop(['Published month','Published quarter','England'],axis=1)

df_r5.rename(columns={'NWD Staff Group':'vacancy_sg','NHS England region':'region_name',
                        'Advertised FTE':'advertised_FTE'},inplace=True)

df_r5 = df_r5.fillna(0)

# Remove region code in brackets
df_r5['region_name'] = df_r5['region_name'].str[:-6].str.rstrip()

# Add staff groupings to match other datasets
df_r5 = pd.merge(df_r5, df_sg_ref, on='vacancy_sg',how='left')

df_r5 = df_r5.drop(['all'],axis=1)

# regional level

df_r5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4226 entries, 0 to 4225
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   month_year      4226 non-null   datetime64[ns]
 1   vacancy_sg      4226 non-null   object        
 2   region_name     4226 non-null   object        
 3   advertised_FTE  4226 non-null   float64       
 4   staff_group     3999 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 165.2+ KB


## Load IV 6 - Reasons for leaving

Data are only quarterly

In [83]:
url_r6 = '../01_data/rfl_dec22.csv'
df_r6 = pd.read_csv(url_r6,parse_dates=['month_year'],dayfirst=True)

#df_r6 = df_r6.drop(['financial_year','quarter'],axis=1)

# df_r5.rename(columns={'NWD Staff Group':'staff_group','NHS England region':'region_name',
#                         'Advertised FTE':'advertised_FTE'},inplace=True)

# df_r5 = df_r5.fillna(0)

# # Remove region code in brackets
# df_r5['region_name'] = df_r5['region_name'].str[:-6].str.rstrip()
# # regional level

# shortern column names and add %_ at beginning
df_r6.rename(columns={'Death in service':'%_death_in_service','Dismissal':'%_dismissal',
                        'End of fixed term':'%_end_of_ft','Flexibility':'%_flexibility',
                        'Health':'%_health','Incompatible working relationships':'%_incompat_relations',
                        'Other':'%_other', 'Pay/Reward':'%_pay_reward', 'Pregnancy':'%_pregnancy',
                        'Progression/CPD':'%_progression_cpd','Relocation':'%_relocation',
                        'Retirement':'%_retirement','Unknown':'%_unknown','Work/Life Balance':'%_work_life_balance',
                        'Workforce Transformation':'%_workforce_transform'},inplace=True)

df_r6['%_other'] = pd.to_numeric(df_r6['%_other'], errors='coerce')
df_r6['%_unknown'] = pd.to_numeric(df_r6['%_unknown'], errors='coerce')
df_r6['%_workforce_transform'] = pd.to_numeric(df_r6['%_workforce_transform'], errors='coerce')

df_r6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   month_year             20 non-null     datetime64[ns]
 1   %_death_in_service     20 non-null     float64       
 2   %_dismissal            20 non-null     float64       
 3   %_end_of_ft            20 non-null     float64       
 4   %_flexibility          20 non-null     float64       
 5   %_health               20 non-null     float64       
 6   %_incompat_relations   20 non-null     float64       
 7   %_other                17 non-null     float64       
 8   %_pay_reward           20 non-null     float64       
 9   %_pregnancy            20 non-null     int64         
 10  %_progression_cpd      20 non-null     float64       
 11  %_relocation           20 non-null     float64       
 12  %_retirement           20 non-null     float64       
 13  %_unkno

## Merge IV dfs to main df

In [84]:
annual_df1.head()

,org_code,region_name,staff_group,month_year,join_HC,join_FTE,leave_HC,leave_FTE,denom_HC,denom_FTE,leaver_rate,joiner_rate
0,R0A,North West,All staff groups,2017-10-01,NaN,NaN,NaN,NaN,19130.0,17095.95832,NaN,NaN
1,R0A,North West,All staff groups,2017-11-01,NaN,NaN,NaN,NaN,19211.0,17171.25859,NaN,NaN
2,R0A,North West,All staff groups,2017-12-01,NaN,NaN,NaN,NaN,19214.0,17170.86679,NaN,NaN
3,R0A,North West,All staff groups,2018-01-01,NaN,NaN,NaN,NaN,19295.0,17240.69400,NaN,NaN
4,R0A,North West,All staff groups,2018-02-01,NaN,NaN,NaN,NaN,19326.0,17271.63829,NaN,NaN


In [85]:
def merge_ivs(df, df_r1, df_r2, df_r3,df_r4,df_r5,df_r6):
    # local unemployment rate
    df1 = pd.merge(df, df_r1, on=['month_year', 'region_name'],how='left')
    df1 = df1.sort_values('month_year')

    # sickness absence
    r2_merge_cols = ['month_year', 'org_code','region_name','staff_group']
    df2 = pd.merge(df1, df_r2, on=r2_merge_cols,how='left')
    df2.drop_duplicates(subset=r2_merge_cols)

    # reason for sickness absence
    r3_merge_cols = ['month_year','staff_group']
    df3 = pd.merge(df2, df_r3, on=r3_merge_cols,how='left')

    # vacancy
    r4_merge_cols = ['month_year','region_name','staff_group']

    df4 = pd.merge(df3, df_r4, on=r4_merge_cols,how='left')

    # df4.drop(columns=['vacancy_sg'], inplace=True)

    # df4['vacancy_rate'] = df4['vacancy_FTE'] / df4['sip_FTE_region']

    # advertised vacancy
    df5 = pd.merge(df4, df_r5, on=r4_merge_cols,how='left')

    # df5.drop(columns=['vacancy_sg'], inplace=True)

    # df5['advertised_rate'] = df5['advertised_FTE'] / df5['sip_FTE_region']

    # df5.drop(columns=['sip_FTE_region','advertised_FTE','vacancy_FTE'], inplace=True)

    # reasons for leaving
    df6 = pd.merge(df5, df_r6, on='month_year',how='left')

    # Need to cut dataframe to earliest and latest data available for all fields. Do this by cutting rows where all values for key variables are zero
    df6 = df6[~((df5['leaver_rate'] == 0) | (df6['joiner_rate'] == 0) | (df6['sickness_absence'] == 0))]
    max_date = df6['month_year'].max()
    min_date = df6['month_year'].min()
    # Calculate the difference in months
    delta = relativedelta(max_date, min_date)
    # Extract the number of months
    months_difference = delta.years * 12 + delta.months
    print(f"After trimming, the dataframe ranges from {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}, "
           f"giving {months_difference} months of data")
    # transform nans into 0s
    df6 = df6.fillna(0)

    # drop duplicates
    df6.drop_duplicates(inplace=True)
    
    return df6
#    return max_date

In [86]:
annual_df_ivs = merge_ivs(annual_df1,df_r1, df_r2, df_r3,df_r4,df_r5,df_r6)

After trimming, the dataframe ranges from 2017-08-01 to 2023-06-01, giving 70 months of data


In [87]:
monthly_df_ivs = merge_ivs(monthly_df1,df_r1, df_r2, df_r3,df_r4,df_r5,df_r6)

After trimming, the dataframe ranges from 2018-07-01 to 2023-06-01, giving 59 months of data


In [92]:
annual_df_ivs.tail()

,org_code,region_name,staff_group,month_year,join_HC,join_FTE,leave_HC,leave_FTE,denom_HC,denom_FTE,...,%_incompat_relations,%_other,%_pay_reward,%_pregnancy,%_progression_cpd,%_relocation,%_retirement,%_unknown,%_work_life_balance,%_workforce_transform
321897,RA4,South West,Midwives,2023-06-01,0.0,0.00000,87.0,70.35333,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321898,RAN,London,Nurses & health visitors,2023-06-01,60.0,57.73333,50.0,45.82999,422.0,385.19611,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321899,RK5,Midlands,All staff groups,2023-06-01,769.0,682.96904,576.0,469.79327,5617.0,4796.56725,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321900,RK5,Midlands,Ambulance staff,2023-06-01,0.0,0.00000,0.0,0.00000,2.0,2.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321901,TAJ,Midlands,"Support to doctors, nurses & midwives",2023-06-01,179.0,159.65094,87.0,76.05333,985.0,858.30291,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
annual_df_ivs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 308764 entries, 0 to 321901
Data columns (total 45 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   org_code                 308764 non-null  object        
 1   region_name              308764 non-null  object        
 2   staff_group              308764 non-null  object        
 3   month_year               308764 non-null  datetime64[ns]
 4   join_HC                  308764 non-null  float64       
 5   join_FTE                 308764 non-null  float64       
 6   leave_HC                 308764 non-null  float64       
 7   leave_FTE                308764 non-null  float64       
 8   denom_HC                 308764 non-null  float64       
 9   denom_FTE                308764 non-null  float64       
 10  leaver_rate              308764 non-null  float64       
 11  joiner_rate              308764 non-null  float64       
 12  local_unemployment   

In [90]:
annual_df_ivs.to_csv(f'annual_dash_data.csv', index=False)

In [91]:
monthly_df_ivs.to_csv(f'monthly_dash_data.csv', index=False)